# Essentially cross_val_score equivalent for tensorflow neural network

### takes model generator, X, Y, fit_params, and cv count

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
%run -i ./Model_Eval.ipynb
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from sklearn.utils import shuffle
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

2023-02-28 22:18:16.398762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 22:18:16.491604: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-28 22:18:16.977244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tychi/miniconda3/lib/
2023-02-28 22:18:16.977325: W tensorflow/compiler/xla/stre

In [2]:
def cv_tf_nn(model_generator, X, Y, fit_params={}, cv=5):
    tqdm._instances.clear()

    assert set(X.index) == set(Y.index)
    full_index = shuffle(list(X.index))

    validation_index_splits = np.array_split(full_index, cv)
    r2_scores = []

    for validation_index in tqdm(validation_index_splits):
        tf.keras.backend.clear_session()
        train_index = shuffle(
            list(set(full_index) - set(validation_index))
        )

        X_train, Y_train = X.loc[train_index], Y.loc[train_index]
        X_val, Y_val = X.loc[validation_index], Y.loc[validation_index]
        
        normalizer = tf.keras.layers.Normalization(axis=-1)
        normalizer.adapt(X_train)

        model = model_generator(normalizer)

        model.fit(
            X_train,
            Y_train,
            validation_data=(X_val, Y_val),
            **fit_params
        )

        # get validation r2_scores
        val_predictions = model.predict(X_val, verbose=0)
        r2_scores.append(
            r2_score(Y_val, val_predictions)
        )
    return r2_scores